## Create an API and a Client

Create an API with 5 endpoints:
* */help* returns with the help/usage of the other 4 endpoints
* 1 endpoint, where one needs to use arguments (key/value pairs)
* 1 endpoint, which returns some data in json format. Either the json should contain a description of what the data is or it should be defined in the /help
* 1 endpoint, which adds extra data to the existing dataset
* 1 endpoint, which replaces data in the existing dataset

The Client should request data from the server and create a table or plot from it.

You can use any dataset for this task. 
E.g. `/home/course/Datasets/owld-datasets/datasets`

You have to prepare for errors or for non-valid queries and need to notify the client about the source of the error!


## How to run this service

* Launch a terminal and enter into the directory, that contains this notebook
* then type
```
preview-nb-api.sh API-worksheet.ipynb 
```
* the first line of output will tell you the URL of the API

In [2]:
import json
import base64
import pandas as pd

In [3]:
# /api/ push_back to url in client!

In [4]:
fname = 'CO2_cc_NOAA_2019.csv'
df = pd.read_csv(fname, parse_dates=[1])#, parse_dates=[1])
# df = df.dropna()
# df =df.rename(columns={'CO2 concentrations (NOAA, 2018)':'CO_2_cc'})
df.Year = pd.to_numeric(df.Year)
#df['Year'] = df['Year'].dt.strftime('%Y')

In [5]:
REQUEST = json.dumps({
 'args' : {}
})

In [6]:
# df.Entity.unique()

In [7]:
# GET /help
help_txt = ("""
<h2>HELP</h2>
<h4>GET /info</h4>
Description: Retrieve either some information about the dataset
Requires: No parameters required
Example query: /api/info
Example result: {"name": "CO2 concentration in the life of earth.", "title": "World CO2 concentration .... }

<h4>GET /data</h4>
Description: Retrieves either the all the data, or the from and until the given dates
Requires: 
<ul>
<li> from (int)</li>
<li> until (int)</li>
</ul>
Returns: 
<ul>
<li> Entity </li>
<li> year</li>
<li> CO_2_cc </li>
</ul>
Example query: /api/data?from=1942&until=1965

<h4>POST /data/</h4>
Description: Adds an extra data point into the dataset for the given 'entity', 'year' with the given 'value'
Requires: 
<ul>
<li> Entity </li>
<li> year (int) </li>
<li> value (float)</li>
</ul>
Returns: 
<ul>
<li> Status 200: upload successful</li>
</ul>
Example query: /api/data?entity=World&year=2016&value=3.9152

<h4>PUT /data/</h4>
Description: Replaces anexisting data point in the dataset for the given 'year' with the given 'value'
Requires: 
<ul>
<li> Entity </li>
<li> year (int) </li>
<li> value (float)</li>
</ul>
Returns: 
<ul>
<li> Status 200: upload successful</li>
</ul>
Example query: /api/data?entity=World&year=2016&value=3.9152


""")
help_txt=help_txt.replace('\n',"")

print(json.dumps(help_txt))

"<h2>HELP</h2><h4>GET /info</h4>Description: Retrieve either some informatin about the datasetRequires: No parameters requiredExample query: /api/infoExample result: {\"name\": \"Global CO2 emissions - CDIAC and UN Population\", \"title\": \"Global CO2 emissions .... }<h4>GET /data</h4>Description: Retrieves either the all the data, or the from and until the given datesRequires: <ul><li> from (int)</li><li> until (int)</li></ul>Returns: <ul><li> year</li><li> metric tons per capita </li></ul>Example query: /api/data?from=1965&until=1988<h4>POST /data/</h4>Description: Adds an extra data point into the dataset for the given 'year' with the given 'value'Requires: <ul><li> year (int) </li><li> value (float)</li></ul>Returns: <ul><li> Status 200: upload successful</li></ul>Example query: /api/data?year=2016&value=3.9152<h4>PUT /data/</h4>Description: Replaces anexisting data point in the dataset for the given 'year' with the given 'value'Requires: <ul><li> year (int) </li><li> value (float

In [7]:
# GET /another
req = json.loads(REQUEST)
args = req['args']

In [24]:
# GET /api/data
req = json.loads(REQUEST)
args = req['args']

if not "from" in args or not "until" in args:
    response = {"Error": "Missing argument. Expecting both 'from' and 'until'"}
    print (json.dumps(response))
else:
    date_from = pd.to_numeric(args["from"][0], errors='coerce')
    date_until = pd.to_numeric(args["until"][0], errors='coerce')
    if date_from >= date_until:
        response = {"queryParams" : query_params_dict, "message" : "'from' should be an earlier year than until"}
        print (json.dumps (response))
    elif (date_from > df.Year.max()) or (date_until<df.Year.min()):
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict,
            "message" : "valid range for 'from' and 'until' is between {} and {} ".format(df['Year'].min(), 
                                                                                          df['Year'].max())
        }
        print (json.dumps (response))
    else:
        rows = df[(df["Year"] >= date_from) &
                             (df["Year"] <= date_until)][["Entity","Year", "CO_2_cc"]].copy()
        data_dict = [{"Entity":entity,"Year" : date, "CO_2_cc" : numbers} for entity,date, numbers in rows.values ]
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict, 
            "data" : data_dict
        }
        print (json.dumps (response))

        
# curl -X GET "https://kooplex-edu.elte.hu/notebook/wfct0p-rere-wfct0p/report/api/data?from=1718&until=1919"

{"Error": "Missing argument. Expecting both 'from' and 'until'"}


In [9]:
## Insert new data with POST

In [27]:
# POST /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "value" in args:
    response = { "Error" : "Missing argument. Expecting both 'date' and 'value'" }
    print (json.dumps (response))
else:
    date = args["date"][0]
    value = float(args["value"][0])
    if (df.Year==date).sum():
        response = {
            "Error" : "'date' should not be an existing value"
            }
        print (json.dumps (response))
    else:
        new_row = {'Entity': 'World',
                   'Year': pd.to_numeric(date, errors='coerce')
                   ,'CO_2_cc': value}
        df = df.append(new_row, ignore_index=True)
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "upload successful"
        }
        print (json.dumps (response))


# curl -X POST "https://kooplex-edu.elte.hu/notebook/wfct0p-rere-wfct0p/report/api/data?date=1718&value=2.324"

{"Error": "Missing argument. Expecting both 'date' and 'value'"}


In [11]:
## Replace existing data with PUT

In [12]:
# PUT /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "value" in args:
    response = {
    "Error" : "Missing argument. Expecting both 'date' and 'value'"
    }
    print (json.dumps (response))
else:
    date = args["date"][0]
    value = float(args["value"][0])
    if not (df.Year==date).sum():
        response = {
            "Error" : "'date' should be an existing one"
            }
        print (json.dumps (response))
    else:
        df.loc[(df.Entity == 'World') & (df.Year == '2016'),'CO_2_cc'] = value
#         df.loc[df.index.max() + 1] = ['World',date,value]
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "Data has been changed",
            "Year": date,
            "New CO_2_cc": value
        }
        print (json.dumps (response))

{"Error": "Missing argument. Expecting both 'date' and 'value'"}


In [13]:
# GET /:nothingfound

print ("You seem lost, try /help")

You seem lost, try /help


In [14]:
# # GET /help
# req = json.loads(REQUEST)
# args = req['args']

# subhelp = {'Message': 'This is a test API. This endpoint can digest the "simple" and "data" arguments'}

# if 'simple' in args:
#     subhelp = {'Message': 'This is a test API and has a simple argument'}
#     response = json.dumps(subhelp)
# elif 'data' in args:
#     N = int(args['data'][0])
#     subhelp = [{'Message': 'This is a test API and this endpoint returns data '}, {'data': list(range(N))}]
#     #subhelp = [{'Message': 'This is a test API and this endpoint returns data '}, {'data': args['data']}]
#     response = json.dumps(subhelp) 
# else:
#     response = json.dumps(subhelp)
    
# print(response)

In [15]:
# # GET /another
# req = json.loads(REQUEST)
# args = req['args']